In [3]:
import pandas as pd
import numpy as np
import re
import argparse
from PyPDF2 import PdfReader
import os
from textcleaningutils import *
import shutil as sh
from multiprocessing import Pool, Manager
import time

/home/marcelo/Escritorio/AsesoriasJT/cultureFinal/textcleaningutils.py:15: SyntaxWarning: invalid escape sequence '\D'
  pattern = re.search(f"[{page_num[0]}][{page_num[1]}]" + "\D" , c).group()
/home/marcelo/Escritorio/AsesoriasJT/cultureFinal/textcleaningutils.py:27: SyntaxWarning: invalid escape sequence '\D'
  pattern = re.search(f"[{page_num[0]}][{page_num[1]}]" + "\D" , c).group()
/home/marcelo/Escritorio/AsesoriasJT/cultureFinal/textcleaningutils.py:42: SyntaxWarning: invalid escape sequence '\D'
  pattern = re.search(f"[{page_num+1}]" + "\D" , c).group()
/home/marcelo/Escritorio/AsesoriasJT/cultureFinal/textcleaningutils.py:55: SyntaxWarning: invalid escape sequence '\D'
  pattern = re.search(f"[{page_num[0]}][{page_num[1]}]" + "\D" , c).group()


In [5]:
doc = "Allegiance Bancshares Inc._Earnings Call_2016-01-26_English.pdf"
reader = PdfReader(f"data/bad_pdf_files/{doc}")
number_of_pages = len(reader.pages)
doc_name = doc.replace("pdf", "txt")
tcfinder = False

In [7]:
for i in range(3):
    page = reader.pages[i]
    temp_text = page.extract_text()
    if "Table of Contents" in temp_text:
        tcfinder = True
        # tcnumber = i
        break
table_contents = temp_text

table_contents = [1 if x in table_contents else 0 for x in ["Call Participants", "Presentation", "Question and Answer"]]
table_contents

[0, 0, 0]

In [10]:
if tcfinder and table_contents[0] == 1 and table_contents[1] == 1:
    participants_num = int(temp_text.split("\n")[-3].replace(".", "")[-1]) - 1
    presentation_num = int(temp_text.split("\n")[-2].replace(".", "")[-1]) - 1
    qa_num = int(temp_text.split("\n")[-1].replace(".", "").split(" ")[-1]) - 1
    quality = 1
elif tcfinder and table_contents[0] == 0 and table_contents[2] == 1:
    participants_num = 0
    presentation_num = int(temp_text.split("\n")[-2].replace(".", "")[-1]) - 1
    qa_num = int(temp_text.split("\n")[-1].replace(".", "").split(" ")[-1]) - 1
    quality = 2
elif tcfinder and table_contents[0] == 0 and table_contents[2] == 0:
    participants_num = 0
    presentation_num = int(temp_text.split("\n")[-1].replace(".", "")[-1]) - 1
    qa_num = 0
    quality = 3
elif not tcfinder:
    page = reader.pages[0]
    temp_text = page.extract_text()
if "Executives" in temp_text:
    table_contents = [2, 2, 2]
    quality = 5
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Call Participants" in temp_text:
            participants_num = i
            break
        else:
            participants_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Presentation" in temp_text:
            presentation_num = i
            break
        else:
            presentation_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Question and Answer" in temp_text:
            qa_num = i
            break
        else:
            qa_num = 99
elif "CORPORATE PARTICIPANTS" in reader.pages[1].extract_text():
    table_contents = [2, 2, 2]
    quality = 6
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Call Participants" in temp_text:
            participants_num = i
            break
        else:
            participants_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "PRESENTATION" in temp_text:
            presentation_num = i
            break
        else:
            presentation_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "QUESTIONS AND ANSWERS" in temp_text:
            qa_num = i
            break
        else:
            qa_num = 99
            
elif "CONFERENCE CALL PARTICIPANTS" in reader.pages[1].extract_text():
    table_contents = [2, 2, 2]
    quality = 7
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "CONFERENCE CALL PARTICIPANTS" in temp_text:
            participants_num = i
            break
        else:
            participants_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "PRESENTATION" in temp_text:
            presentation_num = i
            break
        else:
            presentation_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "QUESTIONS AND ANSWERS" in temp_text:
            qa_num = i
            break
        else:
            qa_num = 99 
else:
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Call Participants" in temp_text:
            participants_num = i
            break
        else:
            participants_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Presentation" in temp_text:
            presentation_num = i
            break
        else:
            presentation_num = 99
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Question and Answer" in temp_text:
            qa_num = i
            break
        else:
            qa_num = 99
    quality = 4
    table_contents = [3, 3, 3]

In [12]:
quality, table_contents

(5, [2, 2, 2])

## Setting Base

In [23]:
if quality in [1, 4]:
    participants = str()
    for i in range(participants_num, presentation_num):
        page = reader.pages[participants_num]
        temp_text = page.extract_text()
        participants = participants + temp_text
    participants = participants.split("\n")
    participants = participants + ["Operator"]
    participants2 = []
    skipper = False
    for i in range(len(participants)):
        if participants[i][-1] != ",":
            if not skipper:
                participants2.append(participants[i])
            skipper = False
        else:
            participants2.append(participants[i] + " " + participants[i + 1])
            skipper = True
    participants = participants2
elif quality == 2:
    participants = []
    for i in range(number_of_pages):
        page = reader.pages[i]
        temp_text = page.extract_text()
        temp_text = temp_text.split("\n")
        for p in temp_text:
            if ("<strong>" in p[:8]) and (p not in participants):
                participants.append(p)
    participants = participants + ["Unknown Speaker"] + ["Operator"] + ["Unknown Operator"]
elif quality == 5:
    mainpage = reader.pages[0]
    participants = mainpage.extract_text()
    participants = participants.split("Presentation")[0].split("Executives")[1].strip().split("\n")
    participants = [x.split("-")[0].strip() for x in participants]
    participants = [x for x in participants if x != "Analysts"]
    participants = participants + ["[Operator Instructions]"] + ["Operator"]
elif quality == 6:
    mainpage = reader.pages[1]
    participants = mainpage.extract_text()
    participants = participants.split("PRESENTATION")[0].split("CORPORATE PARTICIPANTS")[1].strip().split("\n")
    participants = [x.strip() for x in participants]
    participants = [x for x in participants if x != "CONFERENCE CALL PARTICIPANTS"]
    participants = participants + ["[Operator Instructions]"] + ["Operator"] + ["(Operator Instructions)"]
    
elif quality == 7:
    mainpage = reader.pages[1]
    participants = mainpage.extract_text()
    participants = participants.split("Transcript")[0].split("CONFERENCE CALL PARTICIPANTS")[1].strip().split("\n")
    participants = [x.strip() for x in participants]
    participants = [x for x in participants if x != "CONFERENCE CALL PARTICIPANTS"]
    participants = participants + ["[Operator Instructions]"] + ["Operator"] + ["(Operator Instructions)"]

## Processing

In [24]:
quality

5

In [25]:
page = reader.pages[qa_num]
maintext = page.extract_text()
maintext = maintext.replace('[Operator Instructions]', "").strip()
maintext = participantsRemover(maintext, participants)
maintext = firstPageCleaner(maintext)
maintext = '   '.join(maintext)

In [29]:
for i in range(qa_num + 1 , number_of_pages):
    ### Load the Page
    #print(f"========= PROCESSING PAGE {i} ==========")
    page = reader.pages[i]
    currenttext = page.extract_text().strip()
    currenttext = currenttext.replace('[Operator Instructions]' , "").strip()
    if "Disclaimer" not in currenttext:
        ## Check if we are in the Disclaimer Section
        currenttext = participantsRemover(currenttext , participants)
        ## Join Current Text
        currenttext = [x.strip() for x in currenttext]
        currenttext = '   '.join(currenttext).strip()
        #print(currenttext)
        ## Join Whole Text
        if len(maintext) != 0:
            maintext = maintext + "   " + currenttext
        else:
            maintext = maintext + currenttext
        #print("=========== MAINTEXT ========")
        #print(maintext)
    ## If we are in the Disclaimer Section
    else:
        currenttext = currenttext.split("Disclaimer")[0]
        currenttext = participantsRemover(currenttext , participants)
        currenttext = [x.strip() for x in currenttext]
        currenttext = '   '.join(currenttext).strip()
        #print(currenttext)
        if len(maintext) != 0:
            maintext = maintext + "   " + currenttext
        else:
            maintext = maintext + currenttext
        break 

In [30]:
maintext

"Our first question comes from Matt Olney from Stephens.   So first question, just about the overall Houston economy. Given some of the uncertainties within the economy, I'm curious what you're seeing from the competition, whether it's on the smaller community bank side or the larger regional bank side. Any change in the overall level of competition, whether it's on the loan side or on the deposit side?   I'll let Ray address this.   Matt, we've seen the -- on the loan side -- pricing has remained competitive on the loan side from the competition. On the deposit side, we've actually -- we've enjoyed some good deposit growth in the fourth quarter and have yet to see significant competition on the deposit side. But certainly, on the loan side, especially on the C&I space that -- the rates are remaining competitive. We did see 1 bank with its -- increased its rate on CDs here on the 1 year or 14 months CD... [Audio Gap] ...rights. Right. There has been 1 bank on that with time deposits.  

In [ ]:



            
        # ================================ PROCESSING ==============================#
        if qa_num != 99:
            if quality in [1, 2, 4]:
                page = reader.pages[qa_num]
                maintext = page.extract_text()
                maintext = maintext.replace('[Operator Instructions]', "").strip()
                maintext = participantsRemover(maintext, participants)
                maintext = firstPageCleaner(maintext)
                maintext = '   '.join(maintext)
                for i in range(qa_num + 1, number_of_pages - 1):
                    page = reader.pages[i]
                    currenttext = page.extract_text()
                    currenttext = currenttext.replace('[Operator Instructions]', "").strip()
                    currenttext = currenttext.split("All Rights reserved.")[1]
                    currenttext = currenttext.split("\n")
                    currenttext = [x for x in currenttext if len(x) != 0]
                    if i == (number_of_pages - 2):
                        last_strong = 0
                        for h, c in enumerate(currenttext):
                            if "<strong>" in c[:9]:
                                last_strong = h
                        if last_strong != 0:
                            currenttext = currenttext[:last_strong]
                    for p in participants:
                        try:
                            pattern = re.search(p, currenttext[0]).group()
                            currenttext = currenttext[1:]
                        except:
                            currenttext = currenttext
                    currenttext = participantsRemover(currenttext, participants)
                    try:
                        currenttext = numCleanerold(currenttext, i)
                    except:
                        currenttext = currenttext
                    currenttext = '   '.join(currenttext)
                    if len(maintext) != 0:
                        maintext = maintext + "   " + currenttext
                    else:
                        maintext = maintext + currenttext
            elif quality == 5:
                page = reader.pages[qa_num]
                maintext = page.extract_text()
                maintext = maintext.replace('[Operator Instructions]', "").strip()
                maintext = participantsRemover(maintext, participants)
                maintext = firstPageCleaner(maintext)
                maintext = '   '.join(maintext)
                for i in range(qa_num + 1, number_of_pages - 1):
                    page = reader.pages[i]
                    currenttext = page.extract_text()
                    currenttext = currenttext.replace('[Operator Instructions]', "").strip()
                    currenttext = currenttext.split("All Rights reserved.")[1]
                    currenttext = currenttext.split("\n")
                    currenttext = [x for x in currenttext if len(x) != 0]
                    if i == (number_of_pages - 2):
                        last_strong = 0
                        for h, c in enumerate(currenttext):
                            if "<strong>" in c[:9]:
                                last_strong = h
                        if last_strong != 0:
                            currenttext = currenttext[:last_strong]
                    for p in participants:
                        try:
                            pattern = re.search(p, currenttext[0]).group()
                            currenttext = currenttext[1:]
                        except:
                            currenttext = currenttext
                    currenttext = participantsRemover(currenttext, participants)
                    try:
                        currenttext = numCleanerold(currenttext, i)
                    except:
                        currenttext = currenttext
                    currenttext = '   '.join(currenttext)
                    if len(maintext) != 0:
                        maintext = maintext + "   " + currenttext
                    else:
                        maintext = maintext + currenttext
            elif quality in [6,7]:
                page = reader.pages[0]
                currenttext = page.extract_text().strip()
                disclaimer = currenttext.split("affiliated companies")[0] + "affiliated companies."
                page = reader.pages[qa_num]
                maintext = page.extract_text()
                maintext = maintext.replace('(Operator Instructions)' , "").strip()
                maintext = maintext.split("QUESTIONS AND ANSWERS")[1]
                maintext = participantsRemover(maintext , participants)
                maintext = [x.strip() for x in maintext]
                maintext = '   '.join(maintext)
                # for i in range(qa_num + 1, number_of_pages - 1):
                for i in range(qa_num + 1 , number_of_pages):
                    ### Load the Page
                    # print(f"========= PROCESSING PAGE {i} ==========")
                    page = reader.pages[i]
                    currenttext = page.extract_text().strip()
                    currenttext = currenttext.replace('(Operator Instructions)' , "").strip()
                    currenttext = currenttext.split(f"{disclaimer}{i+1}")[1].strip()
                    
                    if "DISCLAIMER" not in currenttext:
                        ## Check if we are in the Disclaimer Section
                        currenttext = participantsRemover(currenttext , participants)
                        ## Join Current Text
                        currenttext = [x.strip() for x in currenttext]
                        currenttext = '   '.join(currenttext).strip()
                        #print(currenttext)
                        ## Join Whole Text
                        if len(maintext) != 0:
                            maintext = maintext + "   " + currenttext
                        else:
                            maintext = maintext + currenttext
                        #print("=========== MAINTEXT ========")
                        #print(maintext)
                    ## If we are in the Disclaimer Section
                    else:
                        currenttext = currenttext.split("DISCLAIMER")[0]
                        currenttext = participantsRemover(currenttext , participants)
                        currenttext = [x.strip() for x in currenttext]
                        currenttext = '   '.join(currenttext).strip()
                        #print(currenttext)
                        if len(maintext) != 0:
                            maintext = maintext + "   " + currenttext
                        else:
                            maintext = maintext + currenttext
                        break

            with open(f"{OUTPUT_FOLDER}/txtfiles/{index}_{doc_name}", "w") as f:
                f.write(maintext)
            corpus_list.append(maintext)
            ids_list.append(str(index) + ".F")
            id2doc_list.append((doc, index))
            # print(ids_list)

        elif qa_num == 99:
            print(f"Error processing {doc}: NO QA!!")
            sh.move(f"{INPUT_FOLDER}/{doc}", NO_QA_PDF_FOLDER)
    except Exception as e:
        print(f"Error processing {doc}: {e}")
        sh.move(f"{INPUT_FOLDER}/{doc}", BAD_PDF_FOLDER)
